In [ ]:
from __future__ import division
import numpy as np
from numpy import linalg as LA
#np.seterr(divide='ignore') # these warnings are usually harmless for this code
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
import os
import scipy.stats as stats
import pyhsmm
from pyhsmm.util.text import progprint_xrange
import pyhsmm.basic.distributions as distributions
import scipy.io as sio
import csv
import copy
import time
import pickle 
from sklearn import preprocessing
from numpy import linalg as LA
import scipy
import math

In [ ]:
#load valide vehicle data
filename = 'device10153_satis.sav'
device10153_satis= pickle.load(open(filename, 'rb'))
print(device10153_satis[0:2])
mul = np.array([[1,0,0,0,0],[0,10,0,0,0],[0,0,1,0,0],[0,0,0,100,0],[0,0,0,0,10]])
device10153_satis[:,2:7] = np.dot(device10153_satis[:,2:7],mul)
print(device10153_satis[0:2])
device10153_trip298_satis = device10153_satis

In [ ]:
#load cluster data
filename = 'statis_after_cluster_95_acc_scaled.sav'
cluster_statis = pickle.load(open(filename, 'rb'))
print(cluster_statis.shape)

In [ ]:
#calculate KL divergence
cluster = 200
nearest = []
kl_value = []
for valid in range(device10153_trip298_satis.shape[0]):
    kl_list = []
    for clus in range(cluster):
        clus_ind = clus + 1
        # cluster 
        m1 = np.array([cluster_statis[clus,0],cluster_statis[clus,1]])
        c1 = np.array([[cluster_statis[clus,2],cluster_statis[clus,4]],[cluster_statis[clus,4],cluster_statis[clus,3]]])
        c1_inv = np.linalg.inv(c1)
        m2 = np.array([device10153_trip298_satis[valid,2],device10153_trip298_satis[valid,3]])
        c2 = np.array([[device10153_trip298_satis[valid,4],device10153_trip298_satis[valid,6]],
                      [device10153_trip298_satis[valid,6],device10153_trip298_satis[valid,5]]])
        c2_squaroot = scipy.linalg.sqrtm(c2)
        c2_inv = np.linalg.inv(c2)
        #tra = np.matrix.trace(c1+c2-2*scipy.linalg.sqrtm(c2_squaroot*c1*c2_squaroot))
        #print(tra)
        
        kl_temp1 = np.matrix.trace(np.dot(c2_inv,c1))
        kl_temp2 = np.dot((m2-m1).T,c2_inv)
        kl_temp2 = np.dot(kl_temp2,(m2-m1))-2
        temp = np.linalg.det(c2)/np.linalg.det(c1)
        print('temp',temp)
        kl_temp3 = math.log(temp)
        kl = kl_temp1 + kl_temp2 + kl_temp3
        kl_list.append(kl)
    kl_array = np.array(kl_list)
    #print(kl_array)
    near = np.argmin(kl_array)+1
    print(kl_array[near-1])
    kl_value.append(kl_array[near-1])
    nearest.append(near)
    #below used for debuging. No other purpose
    #if valid == 2:
     #   break

In [ ]:
# check the nearest in the other direction
cluster = 200
nearest = []
kl_value = []

for clus in range(cluster):
    kl_list = []
    for valid in range(device10153_trip298_satis.shape[0]):
        m1 = np.array([cluster_statis[clus,0],cluster_statis[clus,1]])
        c1 = np.array([[cluster_statis[clus,2],cluster_statis[clus,4]],[cluster_statis[clus,4],cluster_statis[clus,3]]])
        c1_inv = np.linalg.inv(c1)
        m2 = np.array([device10153_trip298_satis[valid,2],device10153_trip298_satis[valid,3]])
        c2 = np.array([[device10153_trip298_satis[valid,4],device10153_trip298_satis[valid,6]],
                      [device10153_trip298_satis[valid,6],device10153_trip298_satis[valid,5]]])
        print(c2)
        c2_squaroot = scipy.linalg.sqrtm(c2)
        c2_inv = np.linalg.inv(c2)
        #tra = np.matrix.trace(c1+c2-2*scipy.linalg.sqrtm(c2_squaroot*c1*c2_squaroot))
        #print(tra)
        
        kl_temp1 = np.matrix.trace(np.dot(c2_inv,c1))
        kl_temp2 = np.dot((m2-m1).T,c2_inv)
        temp = np.linalg.det(c2)/np.linalg.det(c1)
        print('temp',temp)
        kl_temp3 = math.log(temp)
        kl = kl_temp1 + kl_temp2 + kl_temp3
        kl_list.append(kl)
    kl_array = np.array(kl_list)
    #print(kl_array)
    near = np.argmin(kl_array)+1
    print(kl_array[near-1])
    kl_value.append(kl_array[near-1])
    nearest.append(near)
        if valid == 2:
            break

In [ ]:
filename = 'device10153_trip298_kl_nearest.sav'
pickle.dump(nearest, open(filename, 'wb'))

import xlsxwriter

workbook = xlsxwriter.Workbook('device10153_trip298_kl_nearest.xlsx')
worksheet = workbook.add_worksheet()
import pandas as pd

## convert your array into a dataframe
df = pd.DataFrame (np.array(nearest))

## save to xlsx file

filepath = 'device10153_trip298_kl_nearest.xlsx'

df.to_excel(filepath, index=False)

In [ ]:
import xlsxwriter
kl_value = np.array(kl_value)
workbook = xlsxwriter.Workbook('kl_value.xlsx')
worksheet = workbook.add_worksheet()
import pandas as pd

## convert your array into a dataframe
df = pd.DataFrame (kl_value.T)

## save to xlsx file

filepath = 'kl_value.xlsx'

df.to_excel(filepath, index=False)